# Two Account KCL Sample

This workbook provides some utilities around the set up of running a KCL consumer that reads from a stream in one AWS account but does its processing, including capturing KCL metrics and checkpointing in DynamoDB, in another AWS account.

In this workbook, we assume there are two accounts available - a producer account that contains the source stream, and a consumer account that will do the stream consumption. These are configured as AWS credential profiles named 'pa' and 'ca' respectively, and the AWS account numbers are available from the environment as PRODUCER_ACCOUNT_NO and CONSUMER_ACCOUNT_NO, respectively.

## Set Up: Source Account Stream

In [ ]:
import boto3

source_session = boto3.Session(profile_name='pa')

source_kda_client = source_session.client('kinesisanalyticsv2')
source_kinesis = source_session.client('kinesis')
source_iam = source_session.client('iam')
source_kda2 = source_session.client('kinesisanalyticsv2')

### Input Stream

In [ ]:
cis = source_kinesis.create_stream(
    StreamName='ExampleInputStream',
    ShardCount=1
)

print(cis)

## Set Up: Source Account IAM

### Source Policy

In the source account, we need a policy the sink account can consume to give access to the input stream.

In [ ]:
import os
source_account = os.environ['PRODUCER_ACCOUNT_NO']
sink_account = os.environ['CONSUMER_ACCOUNT_NO']

In [ ]:
input_stream_reader_trust_relationship="""{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "AWS":  "arn:aws:iam::""" + sink_account + """:user/ca"
        },
        "Action": "sts:AssumeRole"
      }
    ]
}"""

In [ ]:
input_stream_reader_policy="""{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ReadInputStream",
            "Effect": "Allow",
            "Action": [
                "kinesis:DescribeStream",
                "kinesis:GetRecords",
                "kinesis:GetShardIterator",
                "kinesis:ListShards"
            ],
            "Resource": 
               "arn:aws:kinesis:us-east-1:""" + source_account + """:stream/ExampleInputStream"
        }
    ]
}"""
print(input_stream_reader_policy)

In [ ]:
cr = source_iam.create_role(
    RoleName='KA-Source-Stream-Role',
    Path='/service-role/',
    AssumeRolePolicyDocument=input_stream_reader_trust_relationship
)

print(cr)

In [ ]:
cp = source_iam.create_policy(
    PolicyName='kda-stream-reader',
    Path='/service-role/',
    PolicyDocument=input_stream_reader_policy
)

print(cp)

In [ ]:
ap = source_iam.attach_role_policy(
    RoleName='KA-Source-Stream-Role',
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-stream-reader".format(source_account)
)

## Build and Start Consumer Application

At the root of the source directory, build and package the app, then run it.

```
mvn package
java -jar ./target/kclsample-0.0.1.jar
```

## Generate Stream Input

Use this section of the notebook to write data to the input stream. You should see the corressponding records logged in the output of teh java client.

In [ ]:
import datetime
import json
import random
import boto3

def get_data():
    return {
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': round(random.random() * 100, 2)}


def generate(stream_name, kinesis_client):
    while True:
        data = get_data()
        print(data)
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")

In [ ]:
# Run it
generate("ExampleInputStream", source_kinesis)

## Clean Up

#### IAM

In [ ]:
source_iam.detach_role_policy(
    RoleName='KA-Source-Stream-Role',
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-stream-reader".format(source_account)
)

In [ ]:
source_iam.delete_policy(
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-stream-reader".format(source_account)
)

In [ ]:
source_iam.delete_role(
    RoleName='KA-Source-Stream-Role'
)

#### Streams

In [ ]:
source_kinesis.delete_stream(
    StreamName='ExampleInputStream'
)

#### KCL DynamoDB Checkpoint Table

This is created when the sample KCL consumer is run...

In [ ]:
import boto3

sink_session = boto3.Session(profile_name='ca')
sink_ddb_client = sink_session.client('dynamodb')

In [ ]:
sink_ddb_client.list_tables()

In [ ]:
sink_ddb_client.delete_table(
    TableName='ExampleInputStream'
)

In [ ]:
sink_ddb_client.describe_table(
    TableName='ExampleInputStream'
)